In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

key = os.getenv("GEMINI_KEY")

In [3]:
#@title Setting up the Auth
import os
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
import textwrap

genai.configure(api_key=key)

In [4]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [5]:
models = [m for m in genai.list_models()]
models

[Model(name='models/chat-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 Chat (Legacy)',
       description='A legacy text-only model optimized for chat conversations',
       input_token_limit=4096,
       output_token_limit=1024,
       supported_generation_methods=['generateMessage', 'countMessageTokens'],
       temperature=0.25,
       top_p=0.95,
       top_k=40),
 Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 (Legacy)',
       description='A legacy model that understands text and generates text as an output',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       top_p=0.95,
       top_k=40),
 Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
       display_name='Embedding Gecko',
       description='Obtai

In [6]:
model = genai.GenerativeModel("gemini-pro")
model

 genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
)

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.7,
                             google_api_key=key)

llm

ChatGoogleGenerativeAI(model='gemini-pro', google_api_key=SecretStr('**********'), client= genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
))

In [8]:
RESPONSE_JSON = """
{
    "1" : "question",
    "2" : "question",
    "3" : "question",
    ...
}

"""

In [9]:
TEMPLATE = """
You are an expert question generator. we are conducting an exam on \
{subject} your job is to generate 15 question of type {typ} for that subject.  \
Make sure the questions are not repeated and check all the questions to be conforming the text as well. \
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} Questions \
### RESPONSE_JSON  \ 
{response_json} \

"""

In [10]:
number = 15
subject = "Introduction to Artificial Intelligence"
typ = "Open Quetion"

In [11]:
gen_prompt = ChatPromptTemplate.from_template(TEMPLATE)

output_parser = StrOutputParser()

In [12]:
generation = gen_prompt | llm | output_parser

In [13]:
quiz = generation.invoke({
    "subject" : subject,
    "typ" : typ,
    "number" : number,
    "response_json" : RESPONSE_JSON
})

# to_markdown(response)

In [14]:
print(quiz)

### RESPONSE_JSON

{
    "1": "Describe the key characteristics and components of artificial intelligence (AI) systems.",
    "2": "Explain the different types of AI, such as machine learning, expert systems, and natural language processing.",
    "3": "Discuss the applications of AI in various industries, such as healthcare, finance, and manufacturing.",
    "4": "Explain the ethical implications of AI, including potential biases and privacy concerns.",
    "5": "Describe the history of AI, including major milestones and key figures.",
    "6": "Discuss the role of data in AI systems, including data collection, preprocessing, and labeling.",
    "7": "Explain the concept of machine learning and describe the different types of machine learning algorithms.",
    "8": "Describe the process of training and evaluating machine learning models.",
    "9": "Discuss the importance of model selection and hyperparameter tuning in machine learning.",
    "10": "Explain the concept of deep learnin

In [15]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Question for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz:
{quiz}

Check from an expert English Writer of the above quiz:

Generate the response in the following format:
### RESPONSE_JSON
{response_json}
"""

eval_prompt = ChatPromptTemplate.from_template(TEMPLATE2)

In [16]:
evaluation = eval_prompt | llm | output_parser

In [17]:
quiz_eval = evaluation.invoke({
    "subject" : subject,
    "quiz" : quiz,
    "response_json" : RESPONSE_JSON
})

print(quiz_eval)

**Complexity Analysis:**

The quiz questions cover a broad range of topics in AI and are generally well-structured. However, some questions may require students to have a deeper understanding of AI concepts.

**Updated Quiz Questions:**

### RESPONSE_JSON

{
    "1": "Define artificial intelligence and explain its key characteristics.",
    "2": "Describe the different types of AI, including their strengths and limitations.",
    "3": "Discuss the ethical considerations associated with the development and use of AI.",
    "4": "Explain the role of data in AI systems and the importance of data quality.",
    "5": "Describe the process of training and evaluating machine learning models.",
    "6": "Explain the concept of deep learning and its applications in various fields.",
    "7": "Discuss the benefits and challenges of using AI in healthcare.",
    "8": "Explain the concept of natural language processing and its applications in machine translation.",
    "9": "Describe the different

In [18]:
import re

# Regular expression pattern
pattern = r'\{.*?\}'

# Extracting substring
substring = re.search(pattern, quiz, re.DOTALL)

if substring:
    substring = substring.group()
    print(substring)
else:
    print("No response_json found.")


{
    "1": "Describe the key characteristics and components of artificial intelligence (AI) systems.",
    "2": "Explain the different types of AI, such as machine learning, expert systems, and natural language processing.",
    "3": "Discuss the applications of AI in various industries, such as healthcare, finance, and manufacturing.",
    "4": "Explain the ethical implications of AI, including potential biases and privacy concerns.",
    "5": "Describe the history of AI, including major milestones and key figures.",
    "6": "Discuss the role of data in AI systems, including data collection, preprocessing, and labeling.",
    "7": "Explain the concept of machine learning and describe the different types of machine learning algorithms.",
    "8": "Describe the process of training and evaluating machine learning models.",
    "9": "Discuss the importance of model selection and hyperparameter tuning in machine learning.",
    "10": "Explain the concept of deep learning and describe the 

In [19]:
import json

quiz_json = json.loads(substring)

In [20]:
import pandas as pd

df = pd.DataFrame.from_dict(quiz_json, orient = "index", columns=["Question"])
df.head()

,Question
1,Describe the key characteristics and component...
2,"Explain the different types of AI, such as mac..."
3,Discuss the applications of AI in various indu...
4,"Explain the ethical implications of AI, includ..."
5,"Describe the history of AI, including major mi..."


In [21]:
print(df['Question'][1])

Explain the different types of AI, such as machine learning, expert systems, and natural language processing.


In [22]:
df.to_csv("Questions.csv")

In [23]:
TEMPLATE3 = """
Hey, I need your help reviewing my answer for a question.
grade can be any floating point number

Here's the question: 
{que}

My answer: 
{ans}

Generate the response in the following format:
### Grade 
grade/10
"""

grade_prompt = ChatPromptTemplate.from_template(TEMPLATE3)

In [24]:
grading = grade_prompt | llm | output_parser

In [25]:
que = df['Question'][1]
ans = "Materiality in financial reporting refers to the importance or significance of information to the users of financial statements. Information is considered material if its omission or misstatement could influence the economic decisions of users. The concept of materiality is subjective and depends on factors such as the size, nature, and circumstances of the item. Significance in financial reporting ensures that only relevant information is included in the financial statements, preventing them from being cluttered with trivial details. Materiality guides accountants and auditors in determining what information to disclose and what to omit, helping users focus on key aspects that impact decision-making."

In [26]:
grade = grading.invoke({
    "que" : que,
    "ans" : ans
})

print(grade)

### Grade
0/10


In [27]:
search_word = "Grade\n"

# Find the index where RESPONSE_JSON occurs
index = quiz.find(search_word)

# Extract the substring after RESPONSE_JSON
if index != -1:
    substring = quiz[index + len(search_word):]
    print(substring)
else:
    print("RESPONSE_JSON not found in the string.")


RESPONSE_JSON not found in the string.


In [28]:
import re

In [29]:
pattern = r'\b\d+\b'

match = re.search(pattern, grade)

if match:
    # Extract the integer grade
    grade = int(match.group())
    print("Grade:", grade)
else:
    print("No grade found.")

Grade: 0


In [45]:
def review_grade(que, ans):
    grade = grading.invoke({
    "que" : que,
    "ans" : ans
    })

    pattern = r'\b\d+\b'

    match = re.search(pattern, grade)

    if match:
        # Extract the integer grade
        grade = int(match.group())
        print("Grade:", grade)
    else:
        print("No grade found.")
    
    return grade

In [31]:
def generate_answer(que):
    return model.generate_content(que).text

In [32]:
result = pd.read_csv("Questions.csv")

result

,Unnamed: 0,Question
0,1,Describe the key characteristics and component...
1,2,"Explain the different types of AI, such as mac..."
2,3,Discuss the applications of AI in various indu...
3,4,"Explain the ethical implications of AI, includ..."
4,5,"Describe the history of AI, including major mi..."
5,6,"Discuss the role of data in AI systems, includ..."
6,7,Explain the concept of machine learning and de...
7,8,Describe the process of training and evaluatin...
8,9,Discuss the importance of model selection and ...
9,10,Explain the concept of deep learning and descr...


In [34]:
answers = []

for question in result['Question'][:7]:
    answers.append(generate_answer(question))

answers

["**Key Characteristics of AI Systems:**\n\n* **Intelligent Behavior:** AI systems exhibit capabilities that are typically associated with human intelligence, such as learning, problem-solving, decision-making, and natural language processing.\n* **Data-Driven:** AI algorithms rely heavily on data to train and improve their performance. They can process vast amounts of data efficiently and extract patterns and insights.\n* **Adaptability:** AI systems can adjust their behavior based on changes in the environment or new information. They can learn from their mistakes, refine their models, and adapt to new challenges.\n* **Autonomy:** AI systems can operate independently of human intervention, performing tasks without explicit instructions. However, the level of autonomy varies depending on the specific system.\n* **Scalability:** AI systems can be scaled up or down to handle different levels of complexity and data volumes.\n\n**Components of AI Systems:**\n\n**1. Sensors and Input:**\n*

In [35]:

for question in result['Question'][7:]:
    answers.append(generate_answer(question))

answers

["**Key Characteristics of AI Systems:**\n\n* **Intelligent Behavior:** AI systems exhibit capabilities that are typically associated with human intelligence, such as learning, problem-solving, decision-making, and natural language processing.\n* **Data-Driven:** AI algorithms rely heavily on data to train and improve their performance. They can process vast amounts of data efficiently and extract patterns and insights.\n* **Adaptability:** AI systems can adjust their behavior based on changes in the environment or new information. They can learn from their mistakes, refine their models, and adapt to new challenges.\n* **Autonomy:** AI systems can operate independently of human intervention, performing tasks without explicit instructions. However, the level of autonomy varies depending on the specific system.\n* **Scalability:** AI systems can be scaled up or down to handle different levels of complexity and data volumes.\n\n**Components of AI Systems:**\n\n**1. Sensors and Input:**\n*

In [36]:
result['Answer'] = answers
result

,Unnamed: 0,Question,Answer
0,1,Describe the key characteristics and component...,**Key Characteristics of AI Systems:**\n\n* **...
1,2,"Explain the different types of AI, such as mac...",**Types of Artificial Intelligence**\n\n**1. M...
2,3,Discuss the applications of AI in various indu...,**Healthcare**\n\n* **Disease diagnosis and pr...
3,4,"Explain the ethical implications of AI, includ...",**Ethical Implications of Artificial Intellige...
4,5,"Describe the history of AI, including major mi...",**Early History (Pre-1950s)**\n\n* **1943:** W...
5,6,"Discuss the role of data in AI systems, includ...",**Role of Data in AI Systems**\n\nData forms t...
6,7,Explain the concept of machine learning and de...,**Concept of Machine Learning**\n\nMachine lea...
7,8,Describe the process of training and evaluatin...,**Process of Training and Evaluating Machine L...
8,9,Discuss the importance of model selection and ...,**Model Selection**\n\nModel selection involve...
9,10,Explain the concept of deep learning and descr...,**Concept of Deep Learning**\n\nDeep learning ...


In [37]:
# result['Answer'] = result['Question'].apply(generate_answer)
# result['Answer']

In [38]:
result['Answer'][0]

"**Key Characteristics of AI Systems:**\n\n* **Intelligent Behavior:** AI systems exhibit capabilities that are typically associated with human intelligence, such as learning, problem-solving, decision-making, and natural language processing.\n* **Data-Driven:** AI algorithms rely heavily on data to train and improve their performance. They can process vast amounts of data efficiently and extract patterns and insights.\n* **Adaptability:** AI systems can adjust their behavior based on changes in the environment or new information. They can learn from their mistakes, refine their models, and adapt to new challenges.\n* **Autonomy:** AI systems can operate independently of human intervention, performing tasks without explicit instructions. However, the level of autonomy varies depending on the specific system.\n* **Scalability:** AI systems can be scaled up or down to handle different levels of complexity and data volumes.\n\n**Components of AI Systems:**\n\n**1. Sensors and Input:**\n* 

In [41]:
result['Question'][0]

'Describe the key characteristics and components of artificial intelligence (AI) systems.'

In [47]:
review_grade(result['Question'][0], result['Answer'][0])

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Grade: 9


9